In [10]:
import pandas as pd
import os

In [11]:
%cd /root/wojtek/mro/mro_yolo
%mkdir OID_mid
%cd OID_mid

/root/wojtek/mro/mro_yolo
mkdir: cannot create directory ‘OID_mid’: File exists
/root/wojtek/mro/mro_yolo/OID_mid


In [12]:
# download annotation files

!wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
!wget https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv
!wget https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv

--2023-11-23 01:08:41--  https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘downloader.py.1’

downloader.py.1     100%[===================>]   4.14K  --.-KB/s    in 0s      

2023-11-23 01:08:41 (19.0 MB/s) - ‘downloader.py.1’ saved [4244/4244]

--2023-11-23 01:08:42--  https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.213.123, 142.251.141.59, 216.58.212.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.213.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12064 (12K) [text/csv]
Saving to: ‘oidv7-cla

In [13]:
class_descriptions = pd.read_csv("oidv7-class-descriptions-boxable.csv")

food_label = class_descriptions[class_descriptions["DisplayName"] == "Food"]["LabelName"].values[0]
food_label

'/m/02wbm'

In [14]:
df_val = pd.read_csv("validation-annotations-bbox.csv")
df_val = df_val[df_val["LabelName"] == "/m/02wbm"]
df_val = df_val.sample(500)
df_val

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
132329,6f700bc8da18e376,xclick,/m/02wbm,1,0.018945,1.000000,0.069880,1.000000,0,1,0,0,0
71212,3bd62655fafb1274,xclick,/m/02wbm,1,0.090625,0.889062,0.100000,0.939583,0,0,0,0,1
117888,6340c116815797cb,xclick,/m/02wbm,1,0.000000,1.000000,0.000000,0.931250,1,1,0,0,0
30391,19d14cd8dc17adc0,xclick,/m/02wbm,1,0.194118,0.920588,0.190687,0.824834,1,0,0,0,0
70524,3b5640d56841b0e4,xclick,/m/02wbm,1,0.367257,0.690265,0.547864,0.662739,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63141,3549daaca0d0556f,xclick,/m/02wbm,1,0.000000,1.000000,0.201527,0.374046,1,1,1,0,0
85698,47dba198f9d76f1f,xclick,/m/02wbm,1,0.160938,0.831250,0.014583,0.893750,0,0,0,0,0
163055,89f81119b54c2448,xclick,/m/02wbm,1,0.690625,0.807813,0.562500,0.710417,1,0,0,0,0
83501,46168164054d505e,xclick,/m/02wbm,1,0.360938,0.767188,0.466667,0.645833,0,0,1,0,0


In [ ]:
df_train = pd.read_csv("oidv6-train-annotations-bbox.csv")
df_train = df_train[df_train["LabelName"] == "/m/02wbm"]
df_train = df_train.sample(5000)
df_train

In [ ]:
def xyxy_to_xywh(row):
    x_min, x_max, y_min, y_max = row["XMin"], row["XMax"], row["YMin"], row["YMax"]

    width = x_max - x_min
    height = y_max - y_min

    center_x = (x_min + x_max) / 2
    center_y = (y_min + y_max) / 2

    return center_x, center_y, width, height

In [ ]:
def save_labels(df, path):
    os.makedirs(path, exist_ok=True)

    annotations = {}

    for _, row in df.iterrows():
        x, y, w, h = xyxy_to_xywh(row)
        im_id = row["ImageID"]
        if im_id not in annotations:
            annotations[im_id] = f"0 {x:.4f} {y:.4f} {w:.4f} {h:.4f}"
        else:
            annotations[im_id] += f"\n0 {x:.4f} {y:.4f} {w:.4f} {h:.4f}"

    for im_id, txt in annotations.items():
        filepath = os.path.join(path, str(im_id) + ".txt")
        with open(filepath, "w") as f:
            f.write(txt)

In [ ]:
save_labels(df_val, "labels/val")

In [ ]:
save_labels(df_train, "labels/train")

In [ ]:
with open("val_img_ids.txt", "w") as f:
    img_ids = ["validation/" + im_id for im_id in df_val["ImageID"].unique()]
    txt = "\n".join(img_ids)
    f.write(txt)

In [ ]:
with open("train_img_ids.txt", "w") as f:
    img_ids = ["train/" + im_id for im_id in df_train["ImageID"].unique()]
    txt = "\n".join(img_ids)
    f.write(txt)

In [ ]:
!python downloader.py \
    --download_folder images/val \
    val_img_ids.txt

In [ ]:
!ls images/val | wc

In [24]:
!python downloader.py \
    --download_folder images/train \
    train_img_ids.txt

In [25]:
!ls images/train | wc

   3947    3947   82887


In [122]:
%pwd

'/root/wojtek/mro/mro_yolo/OID_mid'

In [126]:
%%writefile oid.yaml

path: /root/wojtek/mro/mro_yolo/OID_mid
train: images/train
val: images/val
test: 

names:
  0: Food


Overwriting oid.yaml
